# Survey Text Response Analysis

This notebook processes open-ended survey text responses using NLP techniques, sentiment analysis, topic modeling, and keyword extraction. Visualizations and markdown explanations are provided for each step.

## 1. Import Required Libraries
We will use pandas, numpy, nltk, sklearn, matplotlib, seaborn, wordcloud, VADER, and optionally HuggingFace, BERTopic, and KeyBERT.

In [23]:

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.sentiment import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings('ignore')

# Optional imports
try:
    from transformers import pipeline
except ImportError:
    pipeline = None
try:
    from bertopic import BERTopic
except ImportError:
    BERTopic = None
try:
    from keybert import KeyBERT
except ImportError:
    KeyBERT = None
try:
    from rake_nltk import Rake
except ImportError:
    Rake = None

## 2. Load Survey Responses from JSON
We will load the survey responses from the JSON file (`survey-results.json`).

In [24]:
# Load the survey data from JSON file
with open('survey-results-f5ae24e1-9985-450b-b36c-878ffa7f471d.json', 'r', encoding='utf-8') as f:
    survey_data = json.load(f)

# Preview the structure of the JSON data
print('Type:', type(survey_data))
if isinstance(survey_data, dict):
    for k, v in survey_data.items():
        print(f'{k}:', str(v)[:300], '\n')
        break
elif isinstance(survey_data, list):
    print('Sample record:', survey_data[0])

Type: <class 'dict'>
survey: {'survey_id': 'f5ae24e1-9985-450b-b36c-878ffa7f471d', 'topic': 'Student AI/ML session review', 'audience': 'College Student', 'created_at': '2025-08-30T16:56:55.019098+00:00', 'questions_count': 5, 'responses_count': 5} 



## 3. Extract All Text-Based Answers
We will extract all open-ended (text) responses from the survey data for NLP analysis.

In [25]:
# Extract all text-based answers from the survey data
def extract_text_responses(survey_data):
    # Flatten all text answers from all responses
    text_responses = []
    if isinstance(survey_data, dict):
        records = list(survey_data.values())
        if isinstance(records[0], list):
            records = records[0]
    elif isinstance(survey_data, list):
        records = survey_data
    else:
        records = []
    for resp in records:
        if isinstance(resp, dict):
            for k, v in resp.items():
                if isinstance(v, str) and len(v.split()) > 2:  # Heuristic: longer text
                    text_responses.append(v)
    print(f'Extracted {len(text_responses)} text responses.')
    return pd.DataFrame({'response': text_responses})

text_df = extract_text_responses(survey_data)
display(text_df.head())

Extracted 1 text responses.


,response
0,Student AI/ML session review


## 4. NLP Preprocessing
We will preprocess the text responses: lowercasing, stopword removal, and lemmatization.

In [26]:
# Download NLTK resources if not already present
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    tokens = [w.lower() for w in text.split()]
    tokens = [w for w in tokens if w.isalpha() and w not in stop_words]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return ' '.join(tokens)

text_df['processed'] = text_df['response'].apply(preprocess)
display(text_df.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abrarali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Abrarali\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Abrarali\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,response,processed
0,Student AI/ML session review,student session review


## 5. Sentiment Analysis
We will perform sentiment analysis using VADER (rule-based) and optionally a HuggingFace transformer model.

In [27]:
# Ensure tf-keras is installed for compatibility with Transformers and Keras 3
%pip install -q tf-keras

# Sentiment analysis using VADER
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
text_df['vader_sentiment'] = text_df['response'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Optionally, use HuggingFace transformer model for sentiment
if pipeline:
    hf_sentiment = pipeline('sentiment-analysis')
    text_df['hf_sentiment'] = text_df['response'].apply(lambda x: hf_sentiment(x)[0]['label'])
else:
    text_df['hf_sentiment'] = None

display(text_df[['response', 'vader_sentiment', 'hf_sentiment']].head())

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Abrarali\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Note: you may need to restart the kernel to use updated packages.


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english/resolve/714eb0f/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...
Error while downloading from https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english/resolve/714eb0f/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


ValueError: Could not load model distilbert/distilbert-base-uncased-finetuned-sst-2-english with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForSequenceClassification'>, <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\Abrarali\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\response.py", line 779, in _error_catcher
    yield
  File "c:\Users\Abrarali\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\response.py", line 904, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "c:\Users\Abrarali\AppData\Local\Programs\Python\Python313\Lib\site-pa

## 6. Topic Modeling
We will use LDA (Latent Dirichlet Allocation) for topic modeling, and demonstrate BERTopic if available.

In [ ]:
# Topic modeling with LDA
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = vectorizer.fit_transform(text_df['processed'])
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda_topics = lda.fit_transform(dtm)

# Display top words for each topic
def print_top_words(model, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx+1}: ",
              " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print_top_words(lda, vectorizer.get_feature_names_out())

# Optionally, BERTopic
if BERTopic:
    bertopic_model = BERTopic()
    topics, probs = bertopic_model.fit_transform(text_df['processed'])
    text_df['bertopic'] = topics
    print('BERTopic topics:', set(topics))
else:
    text_df['bertopic'] = None

## 7. Keyword Extraction
We will extract keywords from the text responses using RAKE or KeyBERT.

In [ ]:
# Keyword extraction with RAKE or KeyBERT
if Rake:
    rake = Rake()
    text_df['rake_keywords'] = text_df['response'].apply(lambda x: rake.extract_keywords_from_text(x) or rake.get_ranked_phrases())
    print('Sample RAKE keywords:', text_df['rake_keywords'].head().tolist())
elif KeyBERT:
    kw_model = KeyBERT()
    text_df['keybert_keywords'] = text_df['response'].apply(lambda x: kw_model.extract_keywords(x, top_n=5))
    print('Sample KeyBERT keywords:', text_df['keybert_keywords'].head().tolist())
else:
    print('Neither RAKE nor KeyBERT is available.')

## 8. Visualizations
We will visualize frequent terms (WordCloud), sentiment distribution (bar chart), and top topics with sample answers.

In [ ]:
# WordCloud for frequent terms
all_words = ' '.join(text_df['processed'])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_words)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('WordCloud of Frequent Terms')
plt.show()

# Bar chart for sentiment distribution
plt.figure(figsize=(6, 4))
sns.histplot(text_df['vader_sentiment'], bins=20, kde=True)
plt.title('Sentiment Distribution (VADER)')
plt.xlabel('Compound Sentiment Score')
plt.ylabel('Frequency')
plt.show()

# Top topics with sample answers (LDA)
lda_topic_assignments = lda_topics.argmax(axis=1)
text_df['lda_topic'] = lda_topic_assignments
for topic_num in range(lda.n_components):
    print(f'\nTopic {topic_num+1} sample answers:')
    display(text_df[text_df['lda_topic'] == topic_num]['response'].head(3))

## 9. Results and Insights

- **WordCloud** highlights the most frequent terms in open-ended responses.
- **Sentiment analysis** shows the distribution of positive, neutral, and negative feedback.
- **Topic modeling** groups responses into main themes/topics, with sample answers for each.
- **Keyword extraction** surfaces the most important phrases and terms.

These analyses help identify key concerns, suggestions, and overall sentiment from open-ended survey feedback.